# Exercises    
    For each of the following questions, formulate a null and alternative hypothesis (be as specific as you can be), then give an example of what a true positive, true negative, type I and type II errors would look like. Note that some of the questions are intentionally phrased in a vague way. It is your job to reword these as more precise questions that could be tested.

1. Has the network latency gone up since we switched internet service providers?
- null : latency performance is not related to internet providers 
- althypo : latency performance is related to internet providers 
- truepos : avg latency from one company is significantly higher than the other
- trueneg : avg latency from one company is not significantly higher than the other
- type1 : latency is better with one company statistically, but in reality it isnt
- type2 : company doesn't seem to matter with our tests, but in reality actually does
2.  Is the website redesign any good? (just as a comment, what does good even mean? For this I am assuming amt of users visiting site)
- null : website views are not related to the website change
- althypo : website views are related to website change
- truepos : website views are higher with the redesigned group compared to original
- falsepos : website views on original are higher than the redesigned
- type1 : views are higher on the new design, but in actuality it's due to just more people visiting, unrelated to the design
- type2 : our redesign doesn't change our engagement, but in reality it increases user engagement and views. 
3. Is our television ad driving more sales?
- null: advertising campaign does not influence sales
- althypo : advertising campaign influenced sales
- truepos : advertising campaing did create a larger amount of sales
- falsepos : advertising campaign did not increase sales
- type1 : sales are up, but not due to the ad. maybe its the weather
- type2 : sales aren't up, but that's due to people not needing our product. The ad is getting new customers, they just dont increase our sales enough. 

In [9]:
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data

Ace Realty wants to determine whether the average time it takes to sell homes is different for its two offices. A sample of 40 sales from office #1 revealed a mean of 90 days and a standard deviation of 15 days. A sample of 50 sales from office #2 revealed a mean of 100 days and a standard deviation of 20 days. Use a .05 level of significance.



In [3]:
stats.ttest_ind_from_stats(90,15,40,100,20,50, equal_var = False)

Ttest_indResult(statistic=-2.7091418459143854, pvalue=0.00811206270346016)

Load the mpg dataset and use it to answer the following questions:

Is there a difference in fuel-efficiency in cars from 2008 vs 1999?

In [16]:
df = data('mpg')
df = df.rename(columns = {'cty':'city', 'hwy':'highway', 'cyl':'cylinder', 'drv':'drive'})
df['average_mileage'] = df[['highway','city']].mean(axis = 1)
df['mileage_difference'] = df['highway'].sub(df['city'], axis = 0)

In [49]:
old_cars = df[df['year'] == 1999]
new_cars = df[df['year'] == 2008]
stats.levene(old_cars["average_mileage"], new_cars["average_mileage"])

LeveneResult(statistic=0.033228136671080453, pvalue=0.855517569468803)

In [18]:
stats.ttest_ind(old_cars["average_mileage"], new_cars["average_mileage"])

Ttest_indResult(statistic=0.21960177245940962, pvalue=0.8263744040323578)

Are compact cars more fuel-efficient than the average car?

In [34]:
compact_mask = df["class"] == "compact"
compact = df[compact_mask]
avg_car = df[compact_mask == False]

In [21]:
df["class"].unique()

array(['compact', 'midsize', 'suv', '2seater', 'minivan', 'pickup',
       'subcompact'], dtype=object)

In [36]:
stats.levene(compact["average_mileage"], avg_car["average_mileage"])

LeveneResult(statistic=11.107991264810888, pvalue=0.001000307619344077)

In [37]:
stats.ttest_ind(compact["average_mileage"], avg_car["average_mileage"])

Ttest_indResult(statistic=6.731177612837954, pvalue=1.3059121585018135e-10)

Do manual cars get better gas mileage than automatic cars?

In [45]:
auto = df[df['trans'].str.contains("auto")]
manual =  df[df['trans'].str.contains("manual")]

In [47]:
stats.levene(auto["average_mileage"], manual["average_mileage"])

LeveneResult(statistic=0.20075824847529639, pvalue=0.6545276355131857)

In [48]:
stats.ttest_ind(auto["average_mileage"], manual["average_mileage"])

Ttest_indResult(statistic=-4.593437735750014, pvalue=7.154374401145683e-06)